# Assignment 9 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [1]:
#import statements
import numpy as np
import pandas as pd;

In [15]:
#Generate datasaet

#Generate variables
category = pd.Categorical(["C1"] * 4 + ["C2"] * 6)
quality = pd.Categorical(["B"] + ["G"]*3 + ["B"]*2 + ["G"]*4)
temperature = pd.Categorical(["H"] + ["C"] + ["H"]*2 + ["C"]*2 + ["H"]*3 + ["C"])
value = [86, 15, 40, 33, 25, 38, 73, 79, 28, 50]

#Create training set
train_set = pd.DataFrame({"category": category, "quality": quality,\
    "temperature": temperature, "value": value})
train_set

,category,quality,temperature,value
0,C1,B,H,86
1,C1,G,C,15
2,C1,G,H,40
3,C1,G,H,33
4,C2,B,C,25
5,C2,B,C,38
6,C2,G,H,73
7,C2,G,H,79
8,C2,G,H,28
9,C2,G,C,50


In [28]:
#Generate priors